In [140]:
import pandas as pd

In [152]:
key_bid_df = pd.read_excel('wa2_keyword_bid.xlsx')

In [142]:
key_performance_df = pd.read_excel('wa2_keyword_performance.xlsx')

In [143]:
product_db_df = pd.read_excel('wa2_product_database.xlsx')

product_db_df['Breakeven ACoS'] = product_db_df['Breakeven ACoS'].fillna(0.20)
product_db_df['Breakeven Cost'] = product_db_df['Breakeven Cost'].fillna(7.50)

In [144]:
# convert date column to datetime formatting
key_performance_df['date'] = pd.to_datetime(key_performance_df['date'])

# find the most recent date and calculate date range 
end_date = key_performance_df['date'].max()
start_date = end_date - pd.DateOffset(days=30)

# filter the DF based on that date range
filtered_df = key_performance_df[(key_performance_df['date'] >= start_date) & (key_performance_df['date'] <= end_date)]

aggregate_df = filtered_df.groupby(['keyword_id', 'keyword_text']).agg({
    'impressions': 'sum',
    'clicks': 'sum',
    'cost': 'sum',
    'sales': 'sum',
    'units_ordered': 'sum',
    'conversions': 'sum',
    'match_type': 'first',
    'campaign_id': 'first',
    'campaign_name': 'first',
    'short_id': 'first',
    'top_search_multiplier': 'first'
}).reset_index()

aggregate_df['acos'] = (aggregate_df['cost'] / aggregate_df['sales'])

aggregate_df

,keyword_id,keyword_text,impressions,clicks,cost,sales,units_ordered,conversions,match_type,campaign_id,campaign_name,short_id,top_search_multiplier,acos
0,257135855104,50 gallon trash bag,798,51,325.60,2303.98,47,30,exact,142005207010787,434817N - Manual Exact Match (Top Keywords),434817N,10.0,0.141321
1,665395867002,trash bags 39+ gallon,7864,76,78.85,1145.95,41,39,exact,248686931795084,Drawtape 39 Gal (RL3339DT-100) - Manual Exact ...,Drawtape 39 Gal (RL3339DT-100),2.5,0.068808
2,976622689600,a plastic bag,21786,240,422.44,1311.39,69,66,exact,229848649272895,Bio T-Shirt Bags (RL-BIOTY-125) - Video A2x - ...,Bio T-Shirt Bags (RL-BIOTY-125),NaN,0.322131
3,1915220456682,35 gallon trash bag,664,13,69.40,455.92,8,8,exact,79336839833139,404816K - Manual Exact Match (Top Keywords),404816K,10.0,0.152220
4,3642387958158,45 gallon trash bags heavy duty,4203,102,637.31,2598.93,47,40,exact,93698068840771,404816K - Manual Exact Match (Performance),404816K,2.5,0.245220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,144335981199493504,a trash bags,37821,31,87.61,252.95,5,5,exact,144338267903767104,386022K - Manual Exact Match (Organic Ranking)...,386022K,NaN,0.346353
496,144341912666040000,disposable gloves,16043,34,77.75,39.99,1,1,exact,144300634818912608,Hybrid Gloves-M (RL18-10008) - Manual Exact Ma...,Hybrid Gloves-M (RL18-10008),NaN,1.944236
497,144345874365447296,13-gallon trash bags,37443,164,454.92,1798.68,42,41,exact,144155953659193312,243308N - Manual Exact Match (Organic Ranking)...,243308N,NaN,0.252919
498,144356443740833696,garbage bags 13 gallons,14560,42,70.29,349.93,7,6,phrase,144355456894073696,243308N - Manual Phrase Match (Longtail) - Ad ...,243308N,NaN,0.200869


In [145]:
merged_df = pd.merge(aggregate_df, product_db_df, on='short_id', how='left')

merged_aggregate_df = pd.merge(merged_df, key_bid_df, on='keyword_id', how='left')

merged_aggregate_df['effective_bid'] = merged_aggregate_df['bid'] * merged_aggregate_df['top_search_multiplier']

merged_aggregate_df[['keyword_id', 'bid', 'top_search_multiplier', 'effective_bid']]
#merged_df

merged_df = merged_aggregate_df

In [146]:
# Initialize Bid Optimization Rule(s) Applied column
merged_df['Bid Optimization Rule(s) Applied'] = ''

merged_df['Final Proposed Effective Bid'] = merged_df['effective_bid']

#calculate raw bid
final_df['effective_bid'] = final_df['bid'] * final_df['top_search_multiplier']
final_df['raw_bid'] = final_df['bid'] / final_df['top_search_multiplier']

# enforce bid ceiling and floor
top_keywords_condition = final_df['campaign_name'].str.contains('Top Keywords', case=False)
final_df.loc[top_keywords_condition, 'effective_bid'] = final_df.loc[top_keywords_condition, 'effective_bid'].clip(upper=10)
final_df['raw_bid'] = final_df['raw_bid'].clip(lower=0.02)

# Rule 1: Decrease effective bid by 25% if ACoS > (breakeven ACoS + 0.10)
rule1_condition = merged_df['acos'] > (merged_df['Breakeven ACoS'] + 0.10)
merged_df.loc[rule1_condition, 'Final Proposed Effective Bid'] *= 0.75
merged_df.loc[rule1_condition, 'Bid Optimization Rule(s) Applied'] += 'Rule 1, '

# Rule 2: Increase effective bid by 15% if ACoS < (breakeven ACoS - 0.05) AND at least 2 units ordered
rule2_condition = (merged_df['acos'] < (merged_df['Breakeven ACoS'] - 0.05)) & (merged_df['units_ordered'] >= 2)
merged_df.loc[rule2_condition, 'Final Proposed Effective Bid'] *= 1.15
merged_df.loc[rule2_condition, 'Bid Optimization Rule(s) Applied'] += 'Rule 2, '

# Rule 3: Decrease effective bid by 25% if there are no sales and spend > 1.25 * breakeven cost
rule3_condition = (merged_df['sales'] == 0) & (merged_df['cost'] > 1.25 * merged_df['Breakeven Cost'])
merged_df.loc[rule3_condition, 'Final Proposed Effective Bid'] *= 0.75
merged_df.loc[rule3_condition, 'Bid Optimization Rule(s) Applied'] += 'Rule 3, '

#calculate the Final Proposed (Raw) Bid
merged_df['Final Proposed (Raw) Bid'] = merged_df['Final Proposed Effective Bid'] / merged_df['top_search_multiplier']

# init the final proposed state column to active as default
merged_df['Final Proposed State'] = 'enabled'  # Default state is 'enabled'

# Rule 4: Set final proposed state to paused if ACoS > 2.75 * breakeven ACoS
rule4_condition = merged_df['acos'] > 2.75 * merged_df['Breakeven ACoS']
merged_df.loc[rule4_condition, 'Final Proposed State'] = 'paused'
merged_df.loc[rule4_condition, 'Bid Optimization Rule(s) Applied'] += 'Rule 4, '

# Rule 5: Set final proposed state to paused if there are no sales and cost > 2.5 * breakeven cost
rule5_condition = (merged_df['sales'] == 0) & (merged_df['cost'] > 2.5 * merged_df['Breakeven Cost'])
merged_df.loc[rule5_condition, 'Final Proposed State'] = 'paused'
merged_df.loc[rule5_condition, 'Bid Optimization Rule(s) Applied'] += 'Rule 5, '

# remove trailing commas / spaces
merged_df['Bid Optimization Rule(s) Applied'] = merged_df['Bid Optimization Rule(s) Applied'].str.rstrip(', ')

# Print the DataFrame with bid optimization results
merged_df

,keyword_id,keyword_text,impressions,clicks,cost,sales,units_ordered,conversions,match_type,campaign_id,...,AMZ Price,Breakeven Cost,Breakeven ACoS,state,bid,effective_bid,Bid Optimization Rule(s) Applied,Final Proposed Effective Bid,Final Proposed (Raw) Bid,Final Proposed State
0,257135855104,50 gallon trash bag,798,51,325.60,2303.98,47,30,exact,142005207010787,...,52.99,13.3785,0.252472,enabled,1.00,10.0,Rule 2,11.500,1.1500,enabled
1,665395867002,trash bags 39+ gallon,7864,76,78.85,1145.95,41,39,exact,248686931795084,...,27.95,6.6500,0.237925,enabled,3.60,9.0,Rule 2,10.350,4.1400,enabled
2,665395867002,trash bags 39+ gallon,7864,76,78.85,1145.95,41,39,exact,248686931795084,...,25.95,6.7600,0.260501,enabled,3.60,9.0,Rule 2,10.350,4.1400,enabled
3,976622689600,a plastic bag,21786,240,422.44,1311.39,69,66,exact,229848649272895,...,15.95,4.4000,0.294100,enabled,3.00,NaN,,NaN,NaN,enabled
4,1915220456682,35 gallon trash bag,664,13,69.40,455.92,8,8,exact,79336839833139,...,56.99,16.3534,0.286952,enabled,0.95,9.5,Rule 2,10.925,1.0925,enabled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,144335981199493504,a trash bags,37821,31,87.61,252.95,5,5,exact,144338267903767104,...,49.99,11.1119,0.222282,enabled,1.00,NaN,Rule 1,NaN,NaN,enabled
571,144341912666040000,disposable gloves,16043,34,77.75,39.99,1,1,exact,144300634818912608,...,39.99,15.1100,0.377844,enabled,1.00,NaN,"Rule 1, Rule 4",NaN,NaN,paused
572,144345874365447296,13-gallon trash bags,37443,164,454.92,1798.68,42,41,exact,144155953659193312,...,49.99,13.8500,0.277055,enabled,1.00,NaN,,NaN,NaN,enabled
573,144356443740833696,garbage bags 13 gallons,14560,42,70.29,349.93,7,6,phrase,144355456894073696,...,49.99,13.8500,0.277055,enabled,1.00,NaN,Rule 2,NaN,NaN,enabled


In [147]:
final_df = merged_df.drop_duplicates(subset='keyword_id', keep='first')
final_df = final_df.reset_index(drop=True)
final_df

,keyword_id,keyword_text,impressions,clicks,cost,sales,units_ordered,conversions,match_type,campaign_id,...,AMZ Price,Breakeven Cost,Breakeven ACoS,state,bid,effective_bid,Bid Optimization Rule(s) Applied,Final Proposed Effective Bid,Final Proposed (Raw) Bid,Final Proposed State
0,257135855104,50 gallon trash bag,798,51,325.60,2303.98,47,30,exact,142005207010787,...,52.99,13.3785,0.252472,enabled,1.00,10.0,Rule 2,11.500,1.1500,enabled
1,665395867002,trash bags 39+ gallon,7864,76,78.85,1145.95,41,39,exact,248686931795084,...,27.95,6.6500,0.237925,enabled,3.60,9.0,Rule 2,10.350,4.1400,enabled
2,976622689600,a plastic bag,21786,240,422.44,1311.39,69,66,exact,229848649272895,...,15.95,4.4000,0.294100,enabled,3.00,NaN,,NaN,NaN,enabled
3,1915220456682,35 gallon trash bag,664,13,69.40,455.92,8,8,exact,79336839833139,...,56.99,16.3534,0.286952,enabled,0.95,9.5,Rule 2,10.925,1.0925,enabled
4,3642387958158,45 gallon trash bags heavy duty,4203,102,637.31,2598.93,47,40,exact,93698068840771,...,56.99,16.3534,0.286952,enabled,3.60,9.0,,9.000,3.6000,enabled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,144335981199493504,a trash bags,37821,31,87.61,252.95,5,5,exact,144338267903767104,...,49.99,11.1119,0.222282,enabled,1.00,NaN,Rule 1,NaN,NaN,enabled
496,144341912666040000,disposable gloves,16043,34,77.75,39.99,1,1,exact,144300634818912608,...,39.99,15.1100,0.377844,enabled,1.00,NaN,"Rule 1, Rule 4",NaN,NaN,paused
497,144345874365447296,13-gallon trash bags,37443,164,454.92,1798.68,42,41,exact,144155953659193312,...,49.99,13.8500,0.277055,enabled,1.00,NaN,,NaN,NaN,enabled
498,144356443740833696,garbage bags 13 gallons,14560,42,70.29,349.93,7,6,phrase,144355456894073696,...,49.99,13.8500,0.277055,enabled,1.00,NaN,Rule 2,NaN,NaN,enabled


In [148]:
final_df.head()

,keyword_id,keyword_text,impressions,clicks,cost,sales,units_ordered,conversions,match_type,campaign_id,...,AMZ Price,Breakeven Cost,Breakeven ACoS,state,bid,effective_bid,Bid Optimization Rule(s) Applied,Final Proposed Effective Bid,Final Proposed (Raw) Bid,Final Proposed State
0,257135855104,50 gallon trash bag,798,51,325.60,2303.98,47,30,exact,142005207010787,...,52.99,13.3785,0.252472,enabled,1.00,10.0,Rule 2,11.500,1.1500,enabled
1,665395867002,trash bags 39+ gallon,7864,76,78.85,1145.95,41,39,exact,248686931795084,...,27.95,6.6500,0.237925,enabled,3.60,9.0,Rule 2,10.350,4.1400,enabled
2,976622689600,a plastic bag,21786,240,422.44,1311.39,69,66,exact,229848649272895,...,15.95,4.4000,0.294100,enabled,3.00,NaN,,NaN,NaN,enabled
3,1915220456682,35 gallon trash bag,664,13,69.40,455.92,8,8,exact,79336839833139,...,56.99,16.3534,0.286952,enabled,0.95,9.5,Rule 2,10.925,1.0925,enabled
4,3642387958158,45 gallon trash bags heavy duty,4203,102,637.31,2598.93,47,40,exact,93698068840771,...,56.99,16.3534,0.286952,enabled,3.60,9.0,,9.000,3.6000,enabled


In [149]:
final_df[['keyword_id', 'acos', 'Breakeven ACoS', 'units_ordered', 'Final Proposed Effective Bid']]

,keyword_id,acos,Breakeven ACoS,units_ordered,Final Proposed Effective Bid
0,257135855104,0.141321,0.252472,47,11.500
1,665395867002,0.068808,0.237925,41,10.350
2,976622689600,0.322131,0.294100,69,NaN
3,1915220456682,0.152220,0.286952,8,10.925
4,3642387958158,0.245220,0.286952,47,9.000
...,...,...,...,...,...
495,144335981199493504,0.346353,0.222282,5,NaN
496,144341912666040000,1.944236,0.377844,1,NaN
497,144345874365447296,0.252919,0.277055,42,NaN
498,144356443740833696,0.200869,0.277055,7,NaN


In [150]:
final_df['Breakeven ACoS+10'] = final_df['Breakeven ACoS'] + 0.10
final_df['Breakeven ACoS-5'] = final_df['Breakeven ACoS'] - 0.05
final_df['Breakeven ACoSx2.75'] = final_df['Breakeven ACoS'] * 2.75
final_df['Breakeven Costx1.25'] = final_df['Breakeven Cost'] * 1.25
final_df['Breakeven Costx2.5'] = final_df['Breakeven Cost'] * 2.5

desired_columns = ['keyword_id', 'keyword_text', 'campaign_name', 'short_id', 'effective_bid', 'top_search_multiplier',
                   'impressions', 'clicks', 'cost', 'sales', 'acos', 'units_ordered', 'conversions',
                   'Breakeven ACoS', 'Breakeven ACoS+10', 'Breakeven ACoS-5', 'Breakeven ACoSx2.75', 'Breakeven Cost',
                   'Breakeven Costx1.25', 'Breakeven Costx2.5', 'Bid Optimization Rule(s) Applied',
                   'Final Proposed Effective Bid', 'Final Proposed (Raw) Bid', 'Final Proposed State']

df_to_write = final_df[desired_columns]

df_to_write

,keyword_id,keyword_text,campaign_name,short_id,effective_bid,top_search_multiplier,impressions,clicks,cost,sales,...,Breakeven ACoS+10,Breakeven ACoS-5,Breakeven ACoSx2.75,Breakeven Cost,Breakeven Costx1.25,Breakeven Costx2.5,Bid Optimization Rule(s) Applied,Final Proposed Effective Bid,Final Proposed (Raw) Bid,Final Proposed State
0,257135855104,50 gallon trash bag,434817N - Manual Exact Match (Top Keywords),434817N,10.0,10.0,798,51,325.60,2303.98,...,0.352472,0.202472,0.694298,13.3785,16.723125,33.44625,Rule 2,11.500,1.1500,enabled
1,665395867002,trash bags 39+ gallon,Drawtape 39 Gal (RL3339DT-100) - Manual Exact ...,Drawtape 39 Gal (RL3339DT-100),9.0,2.5,7864,76,78.85,1145.95,...,0.337925,0.187925,0.654293,6.6500,8.312500,16.62500,Rule 2,10.350,4.1400,enabled
2,976622689600,a plastic bag,Bio T-Shirt Bags (RL-BIOTY-125) - Video A2x - ...,Bio T-Shirt Bags (RL-BIOTY-125),NaN,NaN,21786,240,422.44,1311.39,...,0.394100,0.244100,0.808775,4.4000,5.500000,11.00000,,NaN,NaN,enabled
3,1915220456682,35 gallon trash bag,404816K - Manual Exact Match (Top Keywords),404816K,9.5,10.0,664,13,69.40,455.92,...,0.386952,0.236952,0.789118,16.3534,20.441750,40.88350,Rule 2,10.925,1.0925,enabled
4,3642387958158,45 gallon trash bags heavy duty,404816K - Manual Exact Match (Performance),404816K,9.0,2.5,4203,102,637.31,2598.93,...,0.386952,0.236952,0.789118,16.3534,20.441750,40.88350,,9.000,3.6000,enabled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,144335981199493504,a trash bags,386022K - Manual Exact Match (Organic Ranking)...,386022K,NaN,NaN,37821,31,87.61,252.95,...,0.322282,0.172282,0.611277,11.1119,13.889875,27.77975,Rule 1,NaN,NaN,enabled
496,144341912666040000,disposable gloves,Hybrid Gloves-M (RL18-10008) - Manual Exact Ma...,Hybrid Gloves-M (RL18-10008),NaN,NaN,16043,34,77.75,39.99,...,0.477844,0.327844,1.039072,15.1100,18.887500,37.77500,"Rule 1, Rule 4",NaN,NaN,paused
497,144345874365447296,13-gallon trash bags,243308N - Manual Exact Match (Organic Ranking)...,243308N,NaN,NaN,37443,164,454.92,1798.68,...,0.377055,0.227055,0.761902,13.8500,17.312500,34.62500,,NaN,NaN,enabled
498,144356443740833696,garbage bags 13 gallons,243308N - Manual Phrase Match (Longtail) - Ad ...,243308N,NaN,NaN,14560,42,70.29,349.93,...,0.377055,0.227055,0.761902,13.8500,17.312500,34.62500,Rule 2,NaN,NaN,enabled


In [151]:
df_to_write.to_excel('final_xlsx_file.xlsx', index=False)